<a href="https://colab.research.google.com/github/MungaiMwangi001/LLMs/blob/main/FT_GPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Prepare the dataset**
- Data is stored in jsonl fprmat  for OpenAI
- Each line is  a separate  json Object

In [1]:
import json

data = [
    {
        "messages": [
            {"role": "system", "content": "You are a teaching assistant for Machine Learning."},
            {"role": "user", "content": "What is machine learning?"},
            {"role": "assistant", "content": "'Tis but the art of teaching machines to think..."}
        ]
    }
]

with open("train.jsonl", "w") as f:
    for row in data:
        f.write(json.dumps(row) + "\n")


**Install OpenAI library and set Api Key**

- The OpenAI key is stored in collabs

In [6]:
!pip install openai

import os
os.environ["OPENAI_API_KEY"] = ""


**Upload the file to OpenAI**

In [7]:
from openai import OpenAI
client = OpenAI()

upload = client.files.create(
    file=open("train.jsonl", "rb"),
    purpose="fine-tune"
)
print(upload)


FileObject(id='file-EHVfZvuXQ3VFdF56HyBk5x', bytes=245, created_at=1754948566, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)


**Create a fine-tuned model**

In [10]:
fine_tune = client.fine_tuning.jobs.create(
    training_file="file-EHVfZvuXQ3VFdF56HyBk5x",
    model="gpt-3.5-turbo"
)
print(fine_tune)


BadRequestError: Error code: 400 - {'error': {'message': 'Fine-tuning jobs cannot be created on an Explore plan. You can upgrade to a paid plan on your billing page: https://platform.openai.com/account/billing/overview', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}

**Monitor the job**

In [11]:
client.fine_tuning.jobs.list(limit=5)
client.fine_tuning.jobs.retrieve("job-id")

NotFoundError: Error code: 404 - {'error': {'message': 'Could not find fine tune: job-id', 'type': 'invalid_request_error', 'param': 'fine_tune_id', 'code': 'fine_tune_not_found'}}

**Test the model**

In [ ]:
completion = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0613:yourname:custom:xyz123",
    messages=[
        {"role": "system", "content": "You are a teaching assistant for Machine Learning."},
        {"role": "user", "content": "Explain overfitting."}
    ]
)
print(completion.choices[0].message)


***DEPLOYMENT***

In [13]:
from fastapi import FastAPI
from pydantic import BaseModel
from openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = ""

client = OpenAI()
app = FastAPI()

class Query(BaseModel):
    question: str

@app.post("/ask")
def ask(query: Query):
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": "You are a teaching assistant for Machine Learning."},
            {"role": "user", "content": query.question}
        ]
    )
    return {"answer": response.choices[0].message.content}


In [15]:
!uvicorn app:app --host 0.0.0.0 --port 8000


ERROR:    Error loading ASGI app. Could not import module "app".
